<a href="https://colab.research.google.com/github/ayushi15092002/mental-health-workload/blob/main/Functional_Connectivity_Feature_Selection_Filter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
import os
from sklearn.model_selection import train_test_split
# plt.style.use('seaborn-colorblind')
# %matplotlib inline
# from feature_selection import filter_method as ft

## Load Dataset

In [ ]:
data = pd.read_excel(r'/content/drive/My Drive/drdo/features/min_max_fun_conn.xlsx')

In [ ]:
data.head(5)

,Coherence,Imaginary Coherence,Phase Locking Value,corrected imaginary PLV,Pairwise Phase Consistency,Phase Lag Index (PLI),Directed Phase Lag Index (DPLI),Weighted Phase Lag Index (WPLI),Debiased estimator of squared WPLI,y
0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,1.0
1,0.0,0.291424,0.0,0.0,0.000636,0.0,0.0,0.0,0.069174,1.0
2,0.0,0.291424,0.0,0.0,0.000636,0.0,0.0,0.0,0.069174,1.0
3,0.0,0.291424,0.0,0.0,0.000636,0.0,0.0,0.0,0.069174,1.0
4,0.0,0.291424,0.0,0.0,0.000636,0.0,0.0,0.0,0.069174,1.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['y'], axis=1), 
                                                    data.y, test_size=0.2,
                                                    random_state=0)
X_train.shape, X_test.shape

((1880, 9), (471, 9))

## Variance method
removing features that show the same value for the majority/all of the observations (constant/quasi-constant features)

In [ ]:
# the original dataset has no constant variable
quasi_constant_feature = constant_feature_detect(data=X_train,threshold=0.9)

0  variables are found to be almost constant


<ipython-input-6-f737976788eb>:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predominant = (data_copy[feature].value_counts() / np.float(


In [ ]:
# lets create a duumy variable that help us do the demonstration
X_train['dummy'] = np.floor(X_train['worst smoothness']*10)
# variable dummy has> 92% of the observations show one value, 1.0
X_train.dummy.value_counts() / np.float(len(X_train))

KeyError: ignored

In [ ]:
quasi_constant_feature = constant_feature_detect(data=X_train,threshold=0.9)
quasi_constant_feature

0  variables are found to be almost constant


<ipython-input-6-f737976788eb>:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  predominant = (data_copy[feature].value_counts() / np.float(


[]

In [ ]:
# drop that variable
X_train.drop(labels=quasi_constant_feature,axis=1,inplace=True)
print(X_train.shape)

(1880, 9)


## Correlation method
remove features that are highly correlated with each other

In [ ]:
corr = corr_feature_detect(data=X_train,threshold=0.9)
# print all the correlated feature groups!
for i in corr:
    print(i,'\n')

    feature1                    feature2      corr
0  Coherence         Phase Locking Value  0.979451
2  Coherence  Pairwise Phase Consistency  0.976376 

                feature1                         feature2      corr
4  Phase Lag Index (PLI)          corrected imaginary PLV  0.967109
7  Phase Lag Index (PLI)  Weighted Phase Lag Index (WPLI)  0.961898 



then we can decide which ones to remove.

## Mutual Information Filter
Mutual information measures how much information the presence/absence of a feature contributes to making the correct prediction on Y.

In [ ]:
# select the top 3 features
mi = mutual_info(X=X_train,y=y_train,select_k=3)
print(mi)

Index(['Coherence', 'Phase Locking Value', 'Pairwise Phase Consistency'], dtype='object')


In [ ]:
# select the top 20% features
mi = mutual_info(X=X_train,y=y_train,select_k=0.2)
print(mi)

Index(['Coherence', 'Phase Locking Value'], dtype='object')


## Chi-Square Filter
Compute chi-squared stats between each non-negative feature and class

In [ ]:
# select the top 3 features
chi = chi_square_test(X=X_train,y=y_train,select_k=3)
print(chi)

Index(['Coherence', 'Phase Locking Value', 'Pairwise Phase Consistency'], dtype='object')


In [ ]:
# select the top 20% features
chi = chi_square_test(X=X_train,y=y_train,select_k=0.2)
print(chi)

Index(['Phase Locking Value', 'Pairwise Phase Consistency'], dtype='object')


## Univariate ROC-AUC or MSE
builds one decision tree per feature, to predict the target, then make predictions and ranks the features according to the machine learning metric (roc-auc or mse)

In [ ]:
uni_roc_auc = univariate_roc_auc(X_train=X_train,y_train=y_train,
                                   X_test=X_test,y_test=y_test,threshold=0.8)
print(uni_roc_auc)

Directed Phase Lag Index (DPLI)       0.540426
Phase Lag Index (PLI)                 0.525633
Debiased estimator of squared WPLI    0.494720
Pairwise Phase Consistency            0.488218
Weighted Phase Lag Index (WPLI)       0.485711
Imaginary Coherence                   0.485501
Phase Locking Value                   0.485048
Coherence                             0.477753
corrected imaginary PLV               0.475820
dtype: float64
0 out of the 9 featues are kept
Series([], dtype: float64)


In [ ]:
uni_mse = univariate_mse(X_train=X_train,y_train=y_train,
                            X_test=X_test,y_test=y_test,threshold=0.4)
print(uni_mse)

Weighted Phase Lag Index (WPLI)       1.089268
Pairwise Phase Consistency            1.048903
Coherence                             1.019205
corrected imaginary PLV               1.008589
Phase Locking Value                   0.987357
Debiased estimator of squared WPLI    0.985209
Imaginary Coherence                   0.980962
Directed Phase Lag Index (DPLI)       0.979396
Phase Lag Index (PLI)                 0.943715
dtype: float64
9 out of the 9 featues are kept
Coherence                             1.019205
Imaginary Coherence                   0.980962
Phase Locking Value                   0.987357
corrected imaginary PLV               1.008589
Pairwise Phase Consistency            1.048903
Phase Lag Index (PLI)                 0.943715
Directed Phase Lag Index (DPLI)       0.979396
Weighted Phase Lag Index (WPLI)       1.089268
Debiased estimator of squared WPLI    0.985209
dtype: float64


In [ ]:
import pandas as pd
import numpy as np
#from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_classif,chi2
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.metrics import roc_auc_score, mean_squared_error

# 2018.11.17 Created by Eamon.Zhang

def constant_feature_detect(data,threshold=0.98):
    """ detect features that show the same value for the 
    majority/all of the observations (constant/quasi-constant features)
    
    Parameters
    ----------
    data : pd.Dataframe
    threshold : threshold to identify the variable as constant
        
    Returns
    -------
    list of variables names
    """
    
    data_copy = data.copy(deep=True)
    quasi_constant_feature = []
    for feature in data_copy.columns:
        predominant = (data_copy[feature].value_counts() / np.float(
                      len(data_copy))).sort_values(ascending=False).values[0]
        if predominant >= threshold:
            quasi_constant_feature.append(feature)
    print(len(quasi_constant_feature),' variables are found to be almost constant')    
    return quasi_constant_feature


def corr_feature_detect(data,threshold=0.8):
    """ detect highly-correlated features of a Dataframe
    Parameters
    ----------
    data : pd.Dataframe
    threshold : threshold to identify the variable correlated
        
    Returns
    -------
    pairs of correlated variables
    """
    
    corrmat = data.corr()
    corrmat = corrmat.abs().unstack() # absolute value of corr coef
    corrmat = corrmat.sort_values(ascending=False)
    corrmat = corrmat[corrmat >= threshold]
    corrmat = corrmat[corrmat < 1] # remove the digonal
    corrmat = pd.DataFrame(corrmat).reset_index()
    corrmat.columns = ['feature1', 'feature2', 'corr']
   
    grouped_feature_ls = []
    correlated_groups = []
    
    for feature in corrmat.feature1.unique():
        if feature not in grouped_feature_ls:
    
            # find all features correlated to a single feature
            correlated_block = corrmat[corrmat.feature1 == feature]
            grouped_feature_ls = grouped_feature_ls + list(
                correlated_block.feature2.unique()) + [feature]
    
            # append the block of features to the list
            correlated_groups.append(correlated_block)
    return correlated_groups


def mutual_info(X,y,select_k=10):
    
#    mi = mutual_info_classif(X,y)
#    mi = pd.Series(mi)
#    mi.index = X.columns
#    mi.sort_values(ascending=False)
    
    if select_k >= 1:
        sel_ = SelectKBest(mutual_info_classif, k=select_k).fit(X,y)
        col = X.columns[sel_.get_support()]
        
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(mutual_info_classif, percentile=select_k*100).fit(X,y)
        col = X.columns[sel_.get_support()]   
        
    else:
        raise ValueError("select_k must be a positive number")
    
    return col
    

# 2018.11.27 edit Chi-square test
def chi_square_test(X,y,select_k=10):
   
    """
    Compute chi-squared stats between each non-negative feature and class.
    This score should be used to evaluate categorical variables in a classification task
    """
    if select_k >= 1:
        sel_ = SelectKBest(chi2, k=select_k).fit(X,y)
        col = X.columns[sel_.get_support()]
    elif 0 < select_k < 1:
        sel_ = SelectPercentile(chi2, percentile=select_k*100).fit(X,y)
        col = X.columns[sel_.get_support()]   
    else:
        raise ValueError("select_k must be a positive number")  
    
    return col
    

def univariate_roc_auc(X_train,y_train,X_test,y_test,threshold):
   
    """
    First, it builds one decision tree per feature, to predict the target
    Second, it makes predictions using the decision tree and the mentioned feature
    Third, it ranks the features according to the machine learning metric (roc-auc or mse)
    It selects the highest ranked features
    """
    roc_values = []
    for feature in X_train.columns:
        clf = DecisionTreeClassifier()
        clf.fit(X_train[feature].to_frame(), y_train)
        y_scored = clf.predict_proba(X_test[feature].to_frame())
        roc_values.append(roc_auc_score(y_test, y_scored, multi_class = 'ovr', average = 'weighted'))
    roc_values = pd.Series(roc_values)
    roc_values.index = X_train.columns
    print(roc_values.sort_values(ascending=False))
    print(len(roc_values[roc_values > threshold]),'out of the %s featues are kept'% len(X_train.columns))
    keep_col = roc_values[roc_values > threshold]
    return keep_col
        
        
def univariate_mse(X_train,y_train,X_test,y_test,threshold):
   
    """
    First, it builds one decision tree per feature, to predict the target
    Second, it makes predictions using the decision tree and the mentioned feature
    Third, it ranks the features according to the machine learning metric (roc-auc or mse)
    It selects the highest ranked features
    """
    mse_values = []
    for feature in X_train.columns:
        clf = DecisionTreeRegressor()
        clf.fit(X_train[feature].to_frame(), y_train)
        y_scored = clf.predict(X_test[feature].to_frame())
        mse_values.append(mean_squared_error(y_test, y_scored))
    mse_values = pd.Series(mse_values)
    mse_values.index = X_train.columns
    print(mse_values.sort_values(ascending=False))
    print(len(mse_values[mse_values > threshold]),'out of the %s featues are kept'% len(X_train.columns))
    keep_col = mse_values[mse_values > threshold]
    return keep_col        